In [37]:
import json
from datetime import datetime, timedelta
import pytz 
import requests

def fetch_data(query):
    """Fetch pairs data from DeX Screener API based on a query."""
    url = f"https://api.dexscreener.com/latest/dex/search?q={query}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Return the JSON response as a dictionary
    else:
        raise Exception("Failed to fetch data from DeX Screener: HTTP Status Code", response.status_code)

def send_telegram_message(bot_token, message):
    """Send a message to a Telegram chat via bot."""
    send_url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {
        'chat_id': "6018455628",
        'text': message,
        'parse_mode': 'Markdown'
    }
    response = requests.post(send_url, data=data)
    return response.json()


def filter_pairs(data, bot_token):
    current_time = datetime.now(pytz.utc)
    recent_pairs = []
    chat_id = "6018455628"
    

    for pair in data['pairs']:
        creation_time = datetime.fromtimestamp(pair['pairCreatedAt'] / 1000, pytz.utc)
        time_difference = current_time - creation_time

        if time_difference < timedelta(hours=72):  # Recent pairs within last 6 hours
            volume_m5 = pair.get('volume', {}).get('m5', 0)
            txns_m5 = pair.get('txns', {}).get('m5', {})
            total_txns_m5 = txns_m5.get('buys', 0) + txns_m5.get('sells', 0)
            liquidity_m5 = pair.get('liquidity', {}).get('usd', 0)

            if volume_m5 > 1 and total_txns_m5 > 1:
                coin_name = pair['baseToken']['name']
                pair_address = pair['pairAddress']
                pair_link = pair['url']
                message = (f"New pair alert: {coin_name}\n"
                           f"Address: {pair_address}\n"
                           f"Link: {pair_link}\n"
                           f"Volume (last 5 mins): {volume_m5}\n"
                           f"Transactions (last 5 mins): {total_txns_m5}\n"
                           f"Liquidity (USD last 5 mins): {liquidity_m5}")
                send_telegram_message(bot_token, message)
                recent_pairs.append(pair)
                
    return recent_pairs
    


# Usage example
bot_token = '7010866436:AAHdBGnzbf_CeUUu4T6fLM6yy15Ygg_GIBk'
data = fetch_data("BASE WETH")  # Assuming you have defined fetch_data() previously
recent_pairs = filter_pairs(data, bot_token)
recent_pairs

[{'chainId': 'base',
  'dexId': 'uniswap',
  'url': 'https://dexscreener.com/base/0x8896c86434eeca20241fe19210a6025b0c4286bb',
  'pairAddress': '0x8896C86434EEca20241FE19210A6025b0C4286bB',
  'labels': ['v2'],
  'baseToken': {'address': '0x93593f8cDE9387F15f9aE2eD1683A7fB3e3bA9C1',
   'name': 'BASE420',
   'symbol': 'B420'},
  'quoteToken': {'address': '0x4200000000000000000000000000000000000006',
   'name': 'Wrapped Ether',
   'symbol': 'WETH'},
  'priceNative': '0.0000007269',
  'priceUsd': '0.002305',
  'txns': {'m5': {'buys': 13, 'sells': 7},
   'h1': {'buys': 215, 'sells': 152},
   'h6': {'buys': 1199, 'sells': 900},
   'h24': {'buys': 1199, 'sells': 900}},
  'volume': {'h24': 2812699.43,
   'h6': 2812699.43,
   'h1': 900306.32,
   'm5': 20117.4},
  'priceChange': {'m5': 0.53, 'h1': 81.34, 'h6': 14525, 'h24': 14525},
  'liquidity': {'usd': 402061.11, 'base': 87180261, 'quote': 63.3721},
  'fdv': 2305918,
  'pairCreatedAt': 1713657273000,
  'info': {'imageUrl': 'https://dd.dexscree